In [38]:
from keras.api.layers import Input, Dense, Flatten, MultiHeadAttention, Concatenate, Add, Permute, Dropout, LayerNormalization, Reshape, Layer
from keras.api.models import Model, Sequential
from keras.api.optimizers import Adam
from keras.api.losses import MeanSquaredError
from keras.api.metrics import MeanAbsoluteError
from keras.api.callbacks import EarlyStopping, ModelCheckpoint
from mne.channels import make_standard_montage, get_builtin_montages
import tensorflow as tf
import numpy as np

def gen_sab(num_channels, time_steps, num_heads, mlp_dim, name, spatial_or_temporal="spatial", dropout_rate=0.1, L=1):
    inp = Input(shape=(num_channels, time_steps))
    embed_dim = time_steps if spatial_or_temporal == "spatial" else num_channels

    norm1 = LayerNormalization(epsilon=1e-6)
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
    dropout1 = Dropout(dropout_rate)
    norm2 = LayerNormalization(epsilon=1e-6)
    mlp = Sequential([
        Dense(mlp_dim, activation='gelu'),
        Dropout(dropout_rate),
        Dense(embed_dim),
        Dropout(dropout_rate)               
    ])

    out = inp
    is_temporal = spatial_or_temporal == "temporal"

    if is_temporal:
        out = Permute((2, 1))(out) # Change to (batch_size, time_steps, channels)
        
    for _ in range(L):
        out_norm = norm1(out)
        attn_output = attn(out_norm, out_norm)
        out1 = out + dropout1(attn_output)

        out1_norm = norm2(out1)
        mlp_output = mlp(out1_norm)
        out2 = out1 + mlp_output

        out += out2 # out = out + out 2
        
    if is_temporal:
        out = Permute((2, 1))(out) # Change back to (batch_size, channels, time_steps)

    return Model(inputs=inp, outputs=out, name=name)

def gen_cab(num_channels, time_steps, num_heads, mlp_dim, name, dropout_rate=0.1, L=1):
    inp = Input(shape=(num_channels, time_steps))

    tsab1 = gen_sab(num_channels, time_steps, num_heads, mlp_dim, "TSAB1", spatial_or_temporal="temporal", dropout_rate=dropout_rate)
    ssab = gen_sab(num_channels, time_steps, num_heads, mlp_dim, "SSAB", spatial_or_temporal="spatial", dropout_rate=dropout_rate)
    tsab2 = gen_sab(num_channels, time_steps, num_heads, mlp_dim, "TSAB2", spatial_or_temporal="temporal", dropout_rate=dropout_rate)

    out = inp
    for _ in range(L):
        out1 = tsab1(out)
        out1 = out + out1

        out2 = ssab(out1)
        out2 = out1 + out2

        out3 = tsab2(out2)
        out3 = out2 + out3
        out += out3

    return Model(inputs=inp, outputs=out, name=name)

def generate_3d_positional_encoding(channel_names, d_model, builtin_montage=None, positions=[]):
    num_channels = len(channel_names)
    builtin_montages = get_builtin_montages()

    if num_channels == 0:
        raise ValueError("The number of channels must be greater than 0.")
    
    if builtin_montage and positions:
        raise ValueError("You can only use either builtin_montage or positions, not both.")
    
    if not builtin_montage and positions and len(positions) != num_channels:
        raise ValueError("The number of positions must match the number of channels.")
    
    if not builtin_montage and positions and len(positions) == num_channels:
        positions = np.array(positions)

    if builtin_montage and not positions and builtin_montage not in builtin_montages:
        raise ValueError(f"Montage '{builtin_montage}' is not available. Please choose from {builtin_montages}.")
    
    if builtin_montage and not positions and builtin_montage in builtin_montages:
        builtin_montage = make_standard_montage(builtin_montage)
        pos_dict = builtin_montage.get_positions()['ch_pos']
        positions = np.array([pos_dict[ch] for ch in channel_names])  # shape: (num_channels, 3)

    assert d_model % 3 == 0, "d_model must be divisible by 3."
    d_model_per_axis = d_model // 3

    pos_encoding = []

    for axis in range(3):
        pos = positions[:, axis]
        pe = np.zeros((num_channels, d_model_per_axis))
        for i in range(d_model_per_axis):
            div_term = np.power(10000.0, (2 * i) / d_model_per_axis)
            pe[:, i] = np.where(i % 2 == 0, np.sin(pos / div_term), np.cos(pos / div_term))
    
        pos_encoding.append(pe)

    pos_encoding = np.concatenate(pos_encoding, axis=-1)    # shape: (num_channels, d_model)
    pos_encoding = np.expand_dims(pos_encoding, axis=0)     # shape: (1, num_channels, d_model)
    return tf.constant(pos_encoding, dtype=tf.float32)      # shape: (1, num_channels, d_model)

def gen_sim(num_channels, time_steps, d_model, num_heads, mlp_dim, low_res_ch_names, high_res_ch_names, name, dropout_rate=0.1, builtin_montage=None, positions=[], L=1):
    dense1 = Dense(d_model, activation='gelu')
    low_res_3d_pos_encoding = generate_3d_positional_encoding(low_res_ch_names, d_model, builtin_montage=builtin_montage, positions=positions)
    cab1 = gen_cab(num_channels, d_model, num_heads, mlp_dim, name="CAB1", dropout_rate=dropout_rate, L=L)
    norm1 = LayerNormalization(epsilon=1e-6)
    mask_token = tf.Variable(initial_value=tf.zeros([1, d_model]), trainable=True)
    dense2 = Dense(d_model, activation='gelu')
    high_res_3d_pos_encoding = generate_3d_positional_encoding(high_res_ch_names, d_model, builtin_montage=builtin_montage, positions=positions)
    cab2 = gen_cab(num_channels, d_model, num_heads, mlp_dim, name="CAB2", dropout_rate=dropout_rate, L=L)
    norm2 = LayerNormalization(epsilon=1e-6)

    inp = Input(shape=(num_channels, time_steps))
    out = inp                                       # channel embedding
    out = dense1(out)
    out = out + low_res_3d_pos_encoding
    
    out = cab1(out)
    out = norm1(out)                            # feature projection
    
    out = Concatenate([out, mask_token])
    
    out = dense2(out)
    out = out + high_res_3d_pos_encoding

    out = cab2(out)
    out = norm2(out)

    return Model(inputs=inp, outputs=out, name=name)

def generate_1d_positional_encoding(time_steps, d_model):
    assert d_model % 2 == 0, "d_model must be even for sin/cos encoding."
    
    pos_encoding = np.zeros((time_steps, d_model))  # Shape: (time_steps, d_model)
    
    for pos in range(time_steps):
        for i in range(d_model // 2):
            div_term = np.power(10000.0, (2 * i) / d_model)
            pos_encoding[pos, 2 * i] = np.sin(pos / div_term)
            pos_encoding[pos, 2 * i + 1] = np.cos(pos / div_term)

    pos_encoding = np.expand_dims(pos_encoding, axis=0)  # Shape: (1, time_steps, d_model)
    return tf.constant(pos_encoding, dtype=tf.float32)

def gen_trm(num_channels, time_steps, d_model, num_heads, mlp_dim, name, dropout_rate=0.1, L=1):
    dense1 = Dense(d_model, activation='gelu')
    _1d_pos_encoding = generate_1d_positional_encoding(time_steps, d_model)
    tsab1 = gen_sab(num_channels, d_model, num_heads, mlp_dim, name="TSAB21", spatial_or_temporal="temporal", dropout_rate=dropout_rate, L=L)
    norm1 = LayerNormalization(epsilon=1e-6)

    dense2 = Dense(d_model, activation='gelu')
    tsab2 = gen_sab(num_channels, d_model, num_heads, mlp_dim, name="TSAB22", spatial_or_temporal="temporal", dropout_rate=dropout_rate, L=L)
    norm2 = LayerNormalization(epsilon=1e-6)

    inp = Input(shape=(num_channels, time_steps))
    out = inp
    # out = Permute((2, 1))(inp)                      # time embedding (batch_size, time_steps, channels)
    out = dense1(out)
    # print(out.shape)
    # print(_1d_pos_encoding.shape)
    out = out + _1d_pos_encoding
    print(out.shape)
    
    out = tsab1(out)
    out = norm1(out)                            # feature projection
    out = dense2(out)
    out = out + _1d_pos_encoding
    out = tsab2(out)
    out = norm2(out)                            # feature projection
    out = Permute((2, 1))(out)                      # reshape to (batch_size, channels, time_steps)
    
    return Model(inputs=inp, outputs=out, name=name)

def gen_model(num_channels, time_steps, d_model, num_heads, mlp_dim, low_res_ch_names, high_res_ch_names, name, dropout_rate=0.1, builtin_montage=None, positions=[], Ls=1, Lt=1):
    high_res_ch_names = ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']
    low_res_ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    builtin_montage = 'standard_1020'
    time_steps = 57
    d_model = 60
    num_heads = 4
    mlp_dim = 128
    dropout_rate = 0.1

    inp = Input(shape=(len(low_res_ch_names), time_steps)) # (batch_size, channels, time_steps)
    sim = gen_sim(num_channels=num_channels, time_steps=time_steps, name="SIM", d_model=d_model, num_heads=num_heads, mlp_dim=mlp_dim, dropout_rate=dropout_rate, low_res_ch_names=low_res_ch_names, high_res_ch_names=high_res_ch_names, builtin_montage=builtin_montage, L=Ls)(inp)
    trm = gen_trm(num_channels=num_channels, time_steps=time_steps, name="TRM", d_model=d_model, num_heads=num_heads, mlp_dim=mlp_dim, dropout_rate=dropout_rate, L=Lt)(sim)
    out = sim + trm

    return Model(inputs=inp, outputs=out, name=name)

In [15]:
# (batch_size, channels, d_model)
gen_sab(num_channels=14, time_steps=60, num_heads=4, mlp_dim=128, name="SSAB", spatial_or_temporal="spatial", dropout_rate=0.1, L=1).summary()

Model: "SSAB"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 14, 60)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 14, 60)    │        120 │ input_layer_12[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 14, 60)    │     58,380 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_26          │ (None, 14, 60)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, 14, 60)    │          0 │ input_layer_12[0… │
│                     │                   │            │ dropout_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 14, 60)    │        120 │ add_14[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_7        │ (None, 14, 60)    │     15,548 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 14, 60)    │          0 │ add_14[0][0],     │
│                     │                   │            │ sequential_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 14, 60)    │          0 │ input_layer_12[0… │
│                     │                   │            │ add_15[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 74,168 (289.72 KB)

 Trainable params: 74,168 (289.72 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
gen_sab(num_channels=14, time_steps=60, num_heads=4, mlp_dim=128, name="TSAB", spatial_or_temporal="temporal", dropout_rate=0.1, L=2).summary()

Model: "TSAB"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_30      │ (None, 14, 60)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_14          │ (None, 60, 14)    │          0 │ input_layer_30[0… │
│ (Permute)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 14)    │         28 │ permute_14[0][0], │
│ (LayerNormalizatio… │                   │            │ add_52[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 14)    │      3,318 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
│                     │                   │            │ layer_normalizat… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_58          │ (None, 60, 14)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_50 (Add)        │ (None, 60, 14)    │          0 │ permute_14[0][0], │
│                     │                   │            │ dropout_58[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 14)    │         28 │ add_50[0][0],     │
│ (LayerNormalizatio… │                   │            │ add_53[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_15       │ (None, 60, 14)    │      3,726 │ layer_normalizat… │
│ (Sequential)        │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_51 (Add)        │ (None, 60, 14)    │          0 │ add_50[0][0],     │
│                     │                   │            │ sequential_15[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_52 (Add)        │ (None, 60, 14)    │          0 │ permute_14[0][0], │
│                     │                   │            │ add_51[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_53 (Add)        │ (None, 60, 14)    │          0 │ add_52[0][0],     │
│                     │                   │            │ dropout_58[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_54 (Add)        │ (None, 60, 14)    │          0 │ add_53[0][0],     │
│                     │                   │            │ sequential_15[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_55 (Add)        │ (None, 60, 14)    │          0 │ add_52[0][0],     │
│                     │                   │            │ add_54[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_15          │ (None, 14, 60)    │          0 │ add_55[0][0]      │
│ (Permute)           │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,100 (27.73 KB)

 Trainable params: 7,100 (27.73 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# gen cab L is not working appropriately
gen_cab(num_channels=14, time_steps=60, num_heads=4, mlp_dim=128, name="CAB", dropout_rate=0.1, L=2).summary()

Model: "CAB"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 14, 60)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TSAB1 (Functional)  │ (None, 14, 60)    │      7,100 │ input_layer_23[0… │
│                     │                   │            │ add_45[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_42 (Add)        │ (None, 14, 60)    │          0 │ input_layer_23[0… │
│                     │                   │            │ TSAB1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ SSAB (Functional)   │ (None, 14, 60)    │     74,168 │ add_42[0][0],     │
│                     │                   │            │ add_46[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_43 (Add)        │ (None, 14, 60)    │          0 │ add_42[0][0],     │
│                     │                   │            │ SSAB[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TSAB2 (Functional)  │ (None, 14, 60)    │      7,100 │ add_43[0][0],     │
│                     │                   │            │ add_47[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_44 (Add)        │ (None, 14, 60)    │          0 │ add_43[0][0],     │
│                     │                   │            │ TSAB2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_45 (Add)        │ (None, 14, 60)    │          0 │ input_layer_23[0… │
│                     │                   │            │ add_44[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_46 (Add)        │ (None, 14, 60)    │          0 │ add_45[0][0],     │
│                     │                   │            │ TSAB1[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_47 (Add)        │ (None, 14, 60)    │          0 │ add_46[0][0],     │
│                     │                   │            │ SSAB[1][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_48 (Add)        │ (None, 14, 60)    │          0 │ add_47[0][0],     │
│                     │                   │            │ TSAB2[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_49 (Add)        │ (None, 14, 60)    │          0 │ add_45[0][0],     │
│                     │                   │            │ add_48[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 88,368 (345.19 KB)

 Trainable params: 88,368 (345.19 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# TODO: the 1d encoding returns (57, 60) and the final channel projection needs to be (channel_count, 57)
gen_trm(num_channels=14, time_steps=57, d_model=60, num_heads=4, mlp_dim=128, name="TRM", dropout_rate=0.1, L=1).summary()

ValueError: Cannot broadcast shape, the failure dim has value 14, which cannot be broadcasted to 57. Input shapes are: [None, 14, 60] and [1, 57, 60].

In [ ]:
# TODO: need to understand the concatenate with the mask token and passing to dense
high_res_ch_names = ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']
low_res_ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
gen_sim(num_channels=14, time_steps=57, d_model=60, num_heads=4, low_res_ch_names=low_res_ch_names, high_res_ch_names=high_res_ch_names, mlp_dim=128, name="SIM", dropout_rate=0.1, L=1, builtin_montage="standard_1020").summary()

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <Concatenate name=concatenate, built=False> (of type <class 'keras.src.layers.merging.concatenate.Concatenate'>)